In [1]:
from google.colab import drive
drive.mount('/content/drive')

root = "/content/drive/MyDrive/aicv/"

Mounted at /content/drive


In [2]:
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgbm
from sklearn.model_selection import KFold
from imblearn.over_sampling import SVMSMOTE

mfcc = pd.read_csv(root + "minmax_train_scaler/train/mfcc_mean_feature.csv")
chroma = pd.read_csv(root + "minmax_train_scaler/train/chroma_mean_feature.csv")
mel = pd.read_csv(root + "minmax_train_scaler/train/mel_mean_feature.csv")
zcr = pd.read_csv(root + "log_and_zcr/log_and_zcr_mean_feature_minmax.csv")

mfcc_private = pd.read_csv(root + "minmax_train_scaler/test/mfcc_mean_feature.csv")
chroma_private = pd.read_csv(root + "minmax_train_scaler/test/chroma_mean_feature.csv")
mel_private = pd.read_csv(root + "minmax_train_scaler/test/mel_mean_feature.csv")
zcr_private = pd.read_csv(root + "log_and_zcr/private_log_and_zcr_mean_feature_minmax.csv")

train_df = pd.concat([zcr.iloc[:, :-1],
                    mfcc.iloc[:, :-1],
                    chroma.iloc[:, :-1], mel], axis=1)
test_df = pd.concat([zcr_private.iloc[:, :-1],
                    mfcc_private.iloc[:, :-1],
                    chroma_private.iloc[:, :-1], mel_private], axis=1)

X, y = train_df.iloc[:, :-1].values, train_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values

print(X.shape, y.shape, X_test.shape)

smote = SVMSMOTE(sampling_strategy=1, random_state=42)

params = {"objective": "binary",
          "boosting_type": "gbdt",
          "metric" : "auc",
          "learning_rate": 0.03,
          "subsample": 0.68,
          "tree_learner": "serial",
          "colsample_bytree": 0.28,
          "early_stopping_rounds": 100,
          "subsample_freq": 1,
          "reg_lambda": 2,
          "reg_alpha": 1,
          "num_leaves": 500,
          "random_state": 42,}

res = np.zeros(X_test.shape[0])
seeds = range(10)
fold_seed = 42
n_splits = 10

for seed in seeds:
    res_cross = np.zeros(X_test.shape[0])

    print("---seed: ", seed)

    kfolds = KFold(n_splits=n_splits, random_state=fold_seed, shuffle=True)

    for f, (trn_idx, val_idx) in enumerate(kfolds.split(X=X, y=y)):

        kf_x_train, kf_y_train = X[trn_idx], y[trn_idx]
        kf_x_train, kf_y_train = smote.fit_resample(kf_x_train, kf_y_train)
        kf_x_valid, kf_y_valid = X[val_idx], y[val_idx]
        
        print(kf_x_train.shape, kf_y_train.shape)
        
        dtrain = lgbm.Dataset(kf_x_train, kf_y_train)
        dvalid = lgbm.Dataset(kf_x_valid, kf_y_valid)
        
        params["random_state"] = seed
        params["num_leaves"] = 500 + seed * 25

        model = lgbm.train(params, 
                          dtrain, 
                          num_boost_round=10000,
                          early_stopping_rounds=params["early_stopping_rounds"],
                          valid_sets=(dtrain, dvalid), 
                          valid_names=("train", "valid"),
                          verbose_eval = 100)

        f = open(root + f"models/model_{seed + 1}_{f + 1}.pkl", "wb")
        pickle.dump(model, f)
        f.close()

        res_cross  += model.predict(X_test)

        print('=' * 80)

    res_cross /= n_splits
    res += res_cross
    
res /= len(seeds)

submission = pd.DataFrame()
submission["uuid"] = test_df["uuid"]
submission["assessment_result"] = res
submission.to_csv("results.csv", index=0)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


(4068, 154) (4068,) (1627, 154)
---seed:  0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6126, 154) (6126,)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.995915	valid's auc: 0.878353
[200]	train's auc: 0.999715	valid's auc: 0.909038
[300]	train's auc: 0.999994	valid's auc: 0.919894
[400]	train's auc: 1	valid's auc: 0.925595
Early stopping, best iteration is:
[338]	train's auc: 1	valid's auc: 0.922409


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

...

(6100, 154) (6100,)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.995841	valid's auc: 0.911422
[200]	train's auc: 0.999636	valid's auc: 0.933159
[300]	train's auc: 0.999992	valid's auc: 0.941063
[400]	train's auc: 1	valid's auc: 0.944768
Early stopping, best iteration is:
[376]	train's auc: 1	valid's auc: 0.944175


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6118, 154) (6118,)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.9958	valid's auc: 0.932219
[200]	train's auc: 0.99963	valid's auc: 0.964349
[300]	train's auc: 0.999991	valid's auc: 0.972237
[400]	train's auc: 1	valid's auc: 0.976292
Early stopping, best iteration is:
[369]	train's auc: 1	valid's auc: 0.975579


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

(6094, 154) (6094,)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.995602	valid's auc: 0.941498
[200]	train's auc: 0.999677	valid's auc: 0.96896
[300]	train's auc: 0.999994	valid's auc: 0.975537
[400]	train's auc: 0.999999	valid's auc: 0.976641
[500]	train's auc: 1	valid's auc: 0.979903
Early stopping, best iteration is:
[463]	train's auc: 1	valid's auc: 0.979061
